In [1]:
from tensorflow.keras.layers import Dense,LSTM,Embedding
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model



In [2]:
words=pd.read_csv("US_Names_data.csv")['name'].drop_duplicates().dropna()

In [3]:
words

0            Madison
1              Emily
2             Hannah
3             Ashley
4            Abigail
             ...    
1917175        Amnen
1917208       Finnan
1917257    Peregrine
1920174       Dierks
1922023        Sully
Name: name, Length: 21118, dtype: object

In [4]:
wordList=list(words)

In [5]:
#### Create a char dectionary 

In [6]:
ord("a")

97

In [7]:
charToInt={"end":0}
IntToChar={0:"end"}

In [8]:
for i in range(26):
    charToInt[chr(i+97)]=(i+1)
    IntToChar[i+1]=chr(i+97)
    

In [44]:
charToInt

{'end': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [9]:
## Integer incoding of all words 

In [10]:
len(wordList)

21118

In [11]:
c=0
finalwordList=[]
for word in wordList:
    if len(word)<=15 and len(word)>2:
        finalwordList.append(word)
        
        
        
   

In [12]:
len(finalwordList)

21067

In [13]:
def Encoding(words):
    result=[]
    for word in words:
        charMapping=[]
        for c in word.lower():
            charMapping.append(charToInt[c])
        charMapping.append(0)
        result.append(charMapping)
    return result 
        
            
            

In [14]:
IntigerembeddedWords=Encoding(finalwordList)

In [15]:
maxLen=max([len(x) for x in IntigerembeddedWords])

In [16]:
maxLen

15

In [17]:
len(IntigerembeddedWords)

21067

In [18]:
IntigerembeddedWords[100]

[3, 8, 18, 9, 19, 20, 9, 14, 1, 0]

In [19]:
### Creating X and Y data for tranning

In [20]:
IntigerembeddedWords[100][:6]

[3, 8, 18, 9, 19, 20]

In [21]:
IntigerembeddedWords[100][1]

8

In [22]:
inputdata=[]
for emmbededword in IntigerembeddedWords:
    for i in range(4,len(emmbededword)+1):
        inputdata.append(emmbededword[:i])

        

In [23]:
padded_sequence=pad_sequences(inputdata,maxlen=maxLen,padding='pre')

In [24]:
padded_sequence.shape

(88063, 15)

In [25]:
padded_sequence

array([[ 0,  0,  0, ...,  1,  4,  9],
       [ 0,  0,  0, ...,  4,  9, 19],
       [ 0,  0,  0, ...,  9, 19, 15],
       ...,
       [ 0,  0,  0, ..., 21, 12, 12],
       [ 0,  0,  0, ..., 12, 12, 25],
       [ 0,  0,  0, ..., 12, 25,  0]], dtype=int32)

In [26]:
X=padded_sequence[:,: -1]

In [27]:
X.shape

(88063, 14)

In [28]:
Y=padded_sequence[:,-1]

In [29]:
Y.shape

(88063,)

In [30]:
X[2]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0, 13,  1,  4,  9, 19],
      dtype=int32)

In [31]:
Y[2]

15

In [32]:
cat_Y=to_categorical(Y,num_classes=27)

In [33]:
cat_Y.shape

(88063, 27)

In [34]:
Train_size=int(len(X)*0.8)

In [35]:
X_train=X[:Train_size+8]
Y_train=cat_Y[:Train_size+8]


In [36]:
X_test=X[Train_size:]
Y_test=cat_Y[Train_size:]

In [37]:
X_test.shape

(17613, 14)

In [38]:
X_train.shape

(70458, 14)

In [39]:
Y_train.shape

(70458, 27)

In [40]:
model=Sequential()
model.add(Embedding(27,10))
model.add(LSTM(150,return_sequences=False,return_state=False))
model.add(Dense(27,activation='softmax'))

In [41]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.build((12,1))

In [45]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 14, 10)         │           270 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 150)            │        96,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 27)             │         4,077 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 302,843 (1.16 MB)

 Trainable params: 100,947 (394.32 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 201,896 (788.66 KB)

In [43]:
model.fit(X_train,Y_train,epochs=200)

Epoch 1/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.2854 - loss: 2.4024
Epoch 2/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.3649 - loss: 2.0380
Epoch 3/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.3942 - loss: 1.9273
Epoch 4/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.4177 - loss: 1.8446
Epoch 5/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.4380 - loss: 1.7643
Epoch 6/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.4544 - loss: 1.7027
Epoch 7/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.4720 - loss: 1.6477
Epoch 8/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.4938 - loss: 1.5899
Epoch 9/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.5050 - loss: 1.5376
Epoch 10/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.5151 - loss: 1.4970
Epoch 11/200
2202/2202 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.5281 - loss: 1.4558
Epoch 12/200
2202

In [379]:
# model.save("model.keras")

In [52]:
model=load_model("model.keras")

In [48]:
def predict(text,model):
    encoded_text=Encoding(text)
    padded_test=pad_sequences([encoded_text[0][:-1]],maxlen=maxLen,padding='pre')
    pridected=model.predict(padded_test,verbose=False)
    top_4CharPred=np.argsort(pridected[0])[-4:]
    top_4Prob=pridected[0][top_4CharPred]

    result={}
    for i in range(4):
        result[top_4Prob[i]]=IntToChar[top_4CharPred[i]]
        
    return result
    
    
    

In [49]:
def completeWord(text,model):
    prediction=predict(text,model)
    probabilitis=sorted(prediction.keys(),reverse=True)
    if prediction[probabilitis[0]]=="end":
        print("Complete name is:-  ",text)
        return
    if probabilitis[0]>0.5:
        text=[text[0]+prediction[probabilitis[0]]]
        completeWord(text,model)
    else:
        completeWord([text[0]+prediction[probabilitis[0]]],model)
        completeWord([text[0]+prediction[probabilitis[1]]],model)
        
        
        
        
        
        
    
    
    
    

In [73]:
word=["vic"]


In [74]:
completeWord(word,model)

Complete name is:-   ['victor']
Complete name is:-   ['vickie']
